In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader, SubsetRandomSampler

In [3]:
import numpy as np
import os
import random
import pandas as pd

In [4]:
from Dataset import Data
from bprModel import BPR

In [5]:
def load_data():    
    df = pd.read_csv('train.csv')
    user_items = {}
    itemId_max=[]
    for i,row in df.iterrows():
        user = int(row[0])
        user_items[user] = [int(x) for x in row[1].split()]
        itemId_max.append(max(user_items[user]))
    num_users = max(user_items.keys())+1
    num_items = max(itemId_max)+1
    return num_users, num_items, user_items

In [6]:
user_size, item_size,user_items = load_data()

In [7]:
batch_size = 3000
epochs = 50
embedding_size = 128

In [ ]:
dataset = Data(user_size, item_size, user_items)

In [9]:
print('The number of train pairs is %d'%len(dataset.train_pair))

The number of train pairs is 304623


In [10]:
loader = DataLoader(dataset, batch_size, shuffle=True)

In [11]:
dataset = Data(user_size, item_size, user_items)
validation_split = .1
shuffle_dataset = True

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(233)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(dataset, batch_size=len(val_indices),
                                                sampler=valid_sampler)

In [12]:
def train(loader, model, optimizer, epochs, batch_size, device):

    model.train()
    model.to(device)
    total_loss = 0.0
    batch_count = 0

    for epoch_id in range(epochs):
        loader.dataset.get_neg()
        for batch_id , (batch_u, batch_i, batch_j)in enumerate(loader):
            
            batch_u = batch_u.to(device)
            batch_i = batch_i.to(device)
            batch_j = batch_j.to(device)
            
            
            optimizer.zero_grad()
            
            loss = model(batch_u, batch_i, batch_j)
            
            loss.backward()

            optimizer.step()
            
            batch_count += 1
            total_loss += loss.data

            avg_loss = total_loss / batch_count
            
            
            if batch_id % 30 == 0:
                print(f"Training Epoch : {epoch_id} | [{batch_id} / {len(loader)}] | Batch Loss = {loss/batch_size:.4f} | Total Average Loss = {avg_loss/batch_size:.4f}\n")
                       

In [13]:
def _eval(model, test_pos, test_sample):
    
    model.eval()
    result = model.predict(test_sample)
    num_users = result.shape[0]

    hit = 0
    ndcg = 0

    for i in range(num_users):
        
        retrieve_items = list(result[i])
        label = test_pos[i]

        if label in retrieve_items:
            hit += 1
            ndcg += (1 / math.log(retrieve_items.index(label)+2,2))

    return (hit / num_users), (ndcg / num_users)

In [14]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")

In [15]:
model = BPR(user_size, item_size, embedding_size, batch_size, device)

In [16]:
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)

In [17]:
train(loader, model, optimizer, epochs, batch_size, device)

Training Epoch : 0 | [0 / 102] | Batch Loss = 0.6931 | Total Average Loss = 0.6931

Training Epoch : 0 | [30 / 102] | Batch Loss = 0.6930 | Total Average Loss = 0.6931

Training Epoch : 0 | [60 / 102] | Batch Loss = 0.6900 | Total Average Loss = 0.6926

Training Epoch : 0 | [90 / 102] | Batch Loss = 0.6588 | Total Average Loss = 0.6880

Training Epoch : 1 | [0 / 102] | Batch Loss = 0.6269 | Total Average Loss = 0.6799

Training Epoch : 1 | [30 / 102] | Batch Loss = 0.5066 | Total Average Loss = 0.6547

Training Epoch : 1 | [60 / 102] | Batch Loss = 0.4250 | Total Average Loss = 0.6176

Training Epoch : 1 | [90 / 102] | Batch Loss = 0.3598 | Total Average Loss = 0.5814

Training Epoch : 2 | [0 / 102] | Batch Loss = 0.3456 | Total Average Loss = 0.5676

Training Epoch : 2 | [30 / 102] | Batch Loss = 0.3274 | Total Average Loss = 0.5392

Training Epoch : 2 | [60 / 102] | Batch Loss = 0.3400 | Total Average Loss = 0.5160

Training Epoch : 2 | [90 / 102] | Batch Loss = 0.3449 | Total Averag

KeyboardInterrupt: 

In [ ]:
w = list(model.parameters())

In [ ]:
user = w[0].detach().numpy()

In [ ]:
item =  w[1].detach().numpy()

In [ ]:
interaction = np.dot(user,item.T)

In [ ]:
predict = pd.DataFrame(columns=['UserId','ItemId'])

In [ ]:
for uid, items in enumerate(interaction):
    for i in user_items[uid]:
        items[i] = -99
    topk = np.argsort(-items)[:50]
    predict.loc[uid,'UserId'] = uid
    predict.loc[uid,'ItemId'] = ' '.join([str(x) for x in topk])

In [ ]:
predict.iloc[0]

In [ ]:
predict.shape

In [ ]:
predict.to_csv('submit_612.csv',index = 0)